<a href="https://colab.research.google.com/gist/oserikov/88cec7c0025e69986a8ce68efd7d45c9/gobot-md-yaml-configs-tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Use RASA DSL to Configure DeepPavlov's GO-Bot

At DeepPavlov, we support a variety of industry-wide and popular standards to support developing Conversational AI solutions.
DSLs, known as Domain-Specific Languages, provide a rich mechanism to define the behavior, or "the what", while,
the underlying system uses the parser to transform these definitions into commands that implement this behavior, or "the how"
using the system's components.
Until very recently we supported two such DSLs, including industry-standard <a href="http://docs.deeppavlov.ai/en/master/features/skills/aiml_skill.html">AIML</a>, as well as <a href="http://docs.deeppavlov.ai/en/master/features/skills/dsl_skill.html">DSL</a> designed by one our partners, EORA.

In this tutorial, you will learn how to use another industrial DSL, or, better said, set of DSLs, introduced by RASA.ai,
to build simple goal-oriented chatbots using DeepPavlov's GO-bot.

As discussed in our blog post, this is the very beginning of our work focused on supporting RASA DSLs as a way to configure DeepPavlov-based
goal-oriented chatbots, and therefore 

To configure a DeepPavlov-based goal-oriented chatbot using these DSLs, you need to have at least three basic config files:
* `stories.md` (or `stories-{trn, tst, val}.md` but these are just subsamples)
* `nlu.md`
* `domain.yml`

These files allow you to define 3 key elements of the chatbot, including product-level stories, NLU training data, and your chatbot's domain.

## Concepts Behind Stories.md, NLU.md, and Domain.yml

### `stories.md`



`stories.md` is a mechanism used to teach your chatbot how to respond to user messages. It allows you to control your chatbot's dialog management.

These "stories" model real conversations between a user and a chatbot. This Markdown-based file is used to define a list of
*stories*, and each *story* can have a list of one or more *intents* with (optional) corresponding *slots*, where each *intent*
has one or more corresponding actions taken by the chatbot.

These actions, in general, can be anything, from simple message replies, to programmable actions that call APIs of other services.
*Note:* In this version, supported actions are limited to simple message replies.

In a way, it can be seen as a *dialogues dataset*.

#### format

Stories file is a markdown file of the following format:

```markdown
## story_title(not used by algorithm, but useful to work with for humans)
* user_action_label{"1st_slot_present_in_action": "slot1_value", .., "Nth_slot_present_in_action": "slotN_value"}
 - system_respective_utterance
* another_user_action_of_the_same_format
  - another_system_response
...

## another_story_title
...

```

**See examples below in this tutorial**

### `nlu.md`


`nlu.md` represents an NLU model of your chatbot. It allows you to provide training examples that show how your chatbot should
understand user messages, and then train a model through these examples.

While DeepPavlov's GO-bot supports JSON-based DSTC-2 format for training data, this Markdown Format introduced by RASA is the easiest one for humans to read and write.

#### format

NLU file is a markdown file of the following format:

```markdown
## intent:possible_user_action_label_1
- An example of user text that has the possible_user_action_label_1 action label
- Another example of user text that has the possible_user_action_label_1 action label
...

## intent:possible_user_action_label_N
- An example of user text that has the (possible_user_action_label_N)[action_label] action label
<!-- Slotfilling dataset is provided as an inline markup of user texts -->
...

```

**See examples below in this tutorial**

### `domain.yml`

`domain.yml` helps you to define the universe your chatbot lives in: what user inputs it expects to get, what actions it should be able to predict,
how to respond, and what information to store.
This YML format is relatively simple, and it can be seen as a dictionary of all components of your chatbot, including but not limited to intents,
actions, responses, and other things.

#### format

Domain file is a YAML file of the following format:

```yaml
# slots section lists the possible slot names (aka slot types) 
# that are used in the domain (i.e. relevant for bot's tasks)
# currently only type: text is supported
slots:
  slot1_name:
    type: text
  ...
  slotN_name:
    type: text

# entities list now follows the slots list 2nd level keys 
# and is present to support upcoming features. Stay tuned for updates with this!
entities:
- slot1_name
...
- slotN_name

# intents section lists the intents that can appear in the stories
# being kept together they do describe the user-side part of go-bot's experience
intents:
  - user_action_label
  - another_user_action_of_the_same_format
  ...

# responses section lists the system response templates.
# Despite system response' titles being usually informative themselves
#   (one could even find them more appropriate when no actual "Natural Language" is needed 
#    (e.g. for buttons actions in bot apps))
# It is though extremely useful to be able to serialize the response title to text. 
# That's what this section content is needed for.
responses:
  system_utterance_1:
    - text: "The text that system responds with"
  another_system_response:
    - text: "Here some text again"

```

**See examples below in this tutorial**

## Basic Chatbot

Let's build the simplest chatbot possible.
This chatbot will be capable of processing three intents: *greeting*, *goodbye*, and *thanks*.

In [1]:
DP_MIN_DEMO_DIR = "dp_minimal_demo_dir"  # we will work in this folder

In [2]:
import os

%cd /content
os.makedirs(DP_MIN_DEMO_DIR, exist_ok=True)
%cd {DP_MIN_DEMO_DIR}

/content
/content/dp_minimal_demo_dir


`stories.md` here is pretty straightforward. In it, you define 3 stories, each having its own intent and reply (utterance).
Take into account the fact that you can combine all of these intents under one story, or add two intents to one stories, and third to another one.

In [3]:
%%writefile stories.md

## greet
* greet
  - utter_greet

## thank
* thank
  - utter_noworries

## goodbye
* bye
  - utter_bye

Writing stories.md


`nlu.md` has an NLU training data that enables DeepPavlov to recognize user phrases as belonging to one the pre-defined intents.

In [4]:
%%writefile nlu.md

## intent:greet
- Hi
- Hey
- Hi bot
- Hey bot
- Hello
- Good morning
- hi again
- hi folks

## intent:bye
- goodbye
- goodnight
- good bye
- good night
- see ya
- toodle-oo
- bye bye
- gotta go
- farewell

## intent:thank
- Thanks
- Thank you
- Thank you so much
- Thanks bot
- Thanks for that
- cheers


Writing nlu.md


`domain.yml` lists the list of: 
* possible user action intents 
* possible system response actions

*Note:* Entities and slots are omitted in this example. See the more sophisticated example below to see how they are defined in the `domain.yml`.

In [5]:
%%writefile domain.yml

intents:
  - greet
  - bye
  - thank

responses:
  utter_noworries:
  - text: No worries!
  utter_greet:
  - text: Hi
  utter_bye:
  - text: Bye!

Writing domain.yml


The next step is to install the `deeppavlov` library.

In [6]:
!pip install git+https://github.com/deepmipt/DeepPavlov.git@feature/gobot-md-yaml-config
!python -m deeppavlov install gobot_simple_dstc2

  Cloning https://github.com/deepmipt/DeepPavlov.git (to revision feature/gobot-md-yaml-config) to /tmp/pip-req-build-ht9t75ir
  Running command git clone -q https://github.com/deepmipt/DeepPavlov.git /tmp/pip-req-build-ht9t75ir
  Running command git checkout -b feature/gobot-md-yaml-config --track origin/feature/gobot-md-yaml-config
  Switched to a new branch 'feature/gobot-md-yaml-config'
  Branch 'feature/gobot-md-yaml-config' set up to track remote branch 'feature/gobot-md-yaml-config' from 'origin'.
     |████████████████████████████████| 51kB 3.7MB/s 
     |████████████████████████████████| 2.1MB 12.1MB/s 
     |████████████████████████████████| 51kB 8.9MB/s 
     |████████████████████████████████| 1.5MB 41.1MB/s 
     |████████████████████████████████| 20.1MB 160kB/s 
     |████████████████████████████████| 10.4MB 55.8MB/s 
     |████████████████████████████████| 512kB 48.5MB/s 
     |████████████████████████████████| 7.3MB 17.1MB/s 
     |████████████████████████████████| 51kB 

2020-07-29 14:36:33.761 INFO in 'deeppavlov.core.common.file'['file'] at line 32: Interpreting 'gobot_simple_dstc2' as '/usr/local/lib/python3.6/dist-packages/deeppavlov/configs/go_bot/gobot_simple_dstc2.json'
     |████████████████████████████████| 10.4MB 5.7MB/s 
     |████████████████████████████████| 2.2MB 56.9MB/s 
  Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4
     |████████████████████████████████| 110.5MB 36kB/s 
     |████████████████████████████████| 512kB 53.4MB/s 
     |████████████████████████████████| 3.8MB 52.3MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=70a99ec04dc3ebce098bf1aec4250f7274f095aeba7f8aae75dee0dddf80e399
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
E

Define the path to our DSL-based configuration files above (the folder we are in right now) and the folder used to store the trained bot.

In [7]:

from deeppavlov import configs
from deeppavlov.core.common.file import read_json


gobot_config = read_json(configs.go_bot.gobot_md_yaml_minimal)

gobot_config['metadata']['variables']['DATA_PATH'] = '.'
gobot_config['metadata']['variables']['MODEL_PATH'] = '.'

!echo "{}" > dstc2-actions2slots.json  # a hack that will be redundant in future versions. It is an empty json file in the data folder.

Since our data is basically the mock tutorial data we will use the same subsamples for all of train, test and valid subsamples. 

However, for a real system you should use different train, test, and valid sample NLU.md files.

In [8]:
!cp stories.md stories-trn.md
!cp stories.md stories-tst.md 
!cp stories.md stories-val.md 

The next step is to train the bot:

In [9]:
from deeppavlov import train_model

train_model(gobot_config, download=True)

2020-07-29 14:37:57.180 INFO in 'deeppavlov.core.data.utils'['utils'] at line 94: Downloading from http://files.deeppavlov.ai/embeddings/glove.6B.100d.txt to /root/.deeppavlov/downloads/embeddings/glove.6B.100d.txt
347MB [00:32, 10.5MB/s]
2020-07-29 14:38:30.843 INFO in 'deeppavlov.download'['download'] at line 132: Skipped http://files.deeppavlov.ai/datasets/gobot_md_yaml_minimal.tar.gz download because of matching hashes
2020-07-29 14:38:30.863 INFO in 'deeppavlov.dataset_readers.md_yaml_dialogs_reader'['md_yaml_dialogs_reader'] at line 123: INSIDE MLU_MD_DialogsDatasetReader._read_domain_knowledge(): reading domain knowledge from /content/dp_minimal_demo_dir/domain.yml
2020-07-29 14:38:30.867 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from /tmp/tmpcurn6nz6]
2020-07-29 14:38:30.869 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from /tmp/tmphq61lzvb]
2020-07-29 14:38:30.871 INFO in '

2020-07-29 14:38:34.266 INFO in 'deeppavlov.dataset_readers.md_yaml_dialogs_reader'['md_yaml_dialogs_reader'] at line 123: INSIDE MLU_MD_DialogsDatasetReader._read_domain_knowledge(): reading domain knowledge from /content/dp_minimal_demo_dir/domain.yml
2020-07-29 14:38:34.270 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from /tmp/tmpdxtbv00d]
2020-07-29 14:38:34.272 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from /tmp/tmpxhizv1vt]
2020-07-29 14:38:34.276 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from /tmp/tmpdswqgy4w]
2020-07-29 14:38:34.375 INFO in 'deeppavlov.models.embedders.glove_embedder'['glove_embedder'] at line 52: [loading GloVe embeddings from `/root/.deeppavlov/downloads/embeddings/glove.6B.100d.txt`]






Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.

Instructions for updating:
Use standard file APIs to check for files with this prefix.


2020-07-29 14:39:10.983 INFO in 'deeppavlov.models.go_bot.policy.policy_network'['policy_network'] at line 89: INSIDE PolicyNetwork init(). Initializing PolicyNetwork from scratch.
2020-07-29 14:39:11.39 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 198: Initial best per_item_action_accuracy of 0.6667


{"valid": {"eval_examples_count": 3, "metrics": {"per_item_action_accuracy": 0.6667}, "time_spent": "0:00:01", "epochs_done": 0, "batches_seen": 0, "train_examples_seen": 0, "impatience": 0, "patience_limit": 10}}


...

2020-07-29 14:39:46.264 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /content/dp_minimal_demo_dir/word.dict]
2020-07-29 14:39:46.265 INFO in 'deeppavlov.dataset_readers.md_yaml_dialogs_reader'['md_yaml_dialogs_reader'] at line 123: INSIDE MLU_MD_DialogsDatasetReader._read_domain_knowledge(): reading domain knowledge from /content/dp_minimal_demo_dir/domain.yml
2020-07-29 14:39:46.271 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from /tmp/tmpmkf5ffbh]
2020-07-29 14:39:46.273 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from /tmp/tmp9h0rcngd]
2020-07-29 14:39:46.275 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from /tmp/tmpwzmh0clp]
2020-07-29 14:39:46.277 INFO in 'deeppavlov.models.embedders.glove_embedder'['glove_embedder'] at line 52: [loading GloVe embeddings from `/root/.deeppavlo

{"valid": {"eval_examples_count": 3, "metrics": {"per_item_action_accuracy": 1.0}, "time_spent": "0:00:01"}}
{"test": {"eval_examples_count": 3, "metrics": {"per_item_action_accuracy": 1.0}, "time_spent": "0:00:01"}}


2020-07-29 14:40:20.543 INFO in 'deeppavlov.models.go_bot.policy.policy_network'['policy_network'] at line 86: INSIDE PolicyNetwork init(). Initializing PolicyNetwork from checkpoint.
2020-07-29 14:40:20.547 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /content/dp_minimal_demo_dir/model/policy]


INFO:tensorflow:Restoring parameters from /content/dp_minimal_demo_dir/model/policy


Chainer[<deeppavlov.models.go_bot.wrapper.DialogComponentWrapper at 0x7f092e14e6a0>,

Finally, it's time to build our bot and experiment with it:

In [10]:
from deeppavlov import build_model
bot = build_model(gobot_config)

2020-07-29 14:40:20.600 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /content/dp_minimal_demo_dir/word.dict]
2020-07-29 14:40:20.603 INFO in 'deeppavlov.dataset_readers.md_yaml_dialogs_reader'['md_yaml_dialogs_reader'] at line 123: INSIDE MLU_MD_DialogsDatasetReader._read_domain_knowledge(): reading domain knowledge from /content/dp_minimal_demo_dir/domain.yml
2020-07-29 14:40:20.607 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from /tmp/tmpptiu0uf8]
2020-07-29 14:40:20.609 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from /tmp/tmpvnyl95qn]
2020-07-29 14:40:20.611 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from /tmp/tmp7jidr5bi]
2020-07-29 14:40:20.613 INFO in 'deeppavlov.models.embedders.glove_embedder'['glove_embedder'] at line 52: [loading GloVe embeddings from `/root/.deeppavlo

INFO:tensorflow:Restoring parameters from /content/dp_minimal_demo_dir/model/policy


In [11]:
bot.reset()

bot(["start"])
bot(["Hi"])[0][0].actions_tuple

('utter_greet',)

Our bot answers with "greeting" to our "greeting". What will it respond to some grateful message?

In [12]:
bot.reset()
bot(["start"])

bot(["Thanks!"])[0][0].actions_tuple

('utter_noworries',)

Ok, "no woories" is an expected response. Let's check if the "goodbye" user message is processed with the corresponding reply:

In [13]:
bot.reset()
bot(["start"])

bot_response_actions = bot(["bye"])[0][0].actions_tuple

In [14]:
import yaml

system_utter2text = yaml.load(open("domain.yml"))["responses"]
system_utter2text[bot_response_actions[0]][0]["text"]

'Bye!'

## More Realistic Demo: Using the DSTSC Schema-Guided Dialogue Dataset

Now let's take the [Schema](asdf) dataset.
We naively converted most of *Restaurants 1* dialogues from Schema to the md+yml format to keep things simple. 

These are the annotated dialogs on the restaurant tables booking domain.
We will not involve the DB usage in our example and the bot should just learn to respond with relevant actions to user's utterances.  

In [15]:
DP_BIG_DEMO_DIR = "dp_big_demo_dir"  # we'll work in this directory

In [16]:
import os

%cd /content
os.makedirs(DP_BIG_DEMO_DIR, exist_ok=True)
%cd {DP_BIG_DEMO_DIR}

/content
/content/dp_big_demo_dir


download the data

In [17]:
# let's get the mentioned converted Schema-dataset subset
!wget http://files.deeppavlov.ai/datasets/schema_resto_md_yaml.tar.gz
!tar -zxf schema_resto_md_yaml.tar.gz 

--2020-07-29 14:40:56--  http://files.deeppavlov.ai/datasets/schema_resto_md_yaml.tar.gz
Resolving files.deeppavlov.ai (files.deeppavlov.ai)... 93.175.29.74
Connecting to files.deeppavlov.ai (files.deeppavlov.ai)|93.175.29.74|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 98725 (96K) [application/octet-stream]
Saving to: ‘schema_resto_md_yaml.tar.gz’

schema_resto_md_yam 100%[===================>]  96.41K   270KB/s    in 0.4s    

2020-07-29 14:40:57 (270 KB/s) - ‘schema_resto_md_yaml.tar.gz’ saved [98725/98725]



First of all let's train the NER model that will be used by bot. No .md or .yml configs illustrated here, but the trained model is inevitable part of the go-bot.

In [18]:
from deeppavlov import configs, train_model
from deeppavlov.core.common.file import read_json

ner_config = read_json(configs.ner.ner_conll2003)
ner_config["dataset_reader"] = read_json(configs.ner.ner_few_shot_ru)["dataset_reader"]
ner_config["dataset_reader"]["data_path"] = "schema_resto_md_yaml/ner"

In [19]:
!python -m deeppavlov download ner_conll2003
!rm -r /root/.deeppavlov/models/ner_conll2003  # remove cached models if present

2020-07-29 14:41:02.348 INFO in 'deeppavlov.core.common.file'['file'] at line 32: Interpreting 'ner_conll2003' as '/usr/local/lib/python3.6/dist-packages/deeppavlov/configs/ner/ner_conll2003.json'
2020-07-29 14:41:03.321 INFO in 'deeppavlov.download'['download'] at line 132: Skipped http://files.deeppavlov.ai/embeddings/glove.6B.100d.txt?config=ner_conll2003 download because of matching hashes
2020-07-29 14:41:03.669 INFO in 'deeppavlov.core.data.utils'['utils'] at line 94: Downloading from http://files.deeppavlov.ai/deeppavlov_data/ner_conll2003_v5.tar.gz?config=ner_conll2003 to /root/.deeppavlov/ner_conll2003_v5.tar.gz
100% 3.12M/3.12M [00:01<00:00, 2.26MB/s]
2020-07-29 14:41:05.413 INFO in 'deeppavlov.core.data.utils'['utils'] at line 269: Extracting /root/.deeppavlov/ner_conll2003_v5.tar.gz archive into /root/.deeppavlov/models


In [20]:
# if rm says that these files are missing that's ok 
#   (we just deleted the whole folder containing these paths ^__^)
# but if they're present we should delete them 
# cause they're checkpoints of some other model
!rm /root/.deeppavlov/models/ner_conll2003/{checkpoint,model_no_pos}*

rm: cannot remove '/root/.deeppavlov/models/ner_conll2003/checkpoint*': No such file or directory
rm: cannot remove '/root/.deeppavlov/models/ner_conll2003/model_no_pos*': No such file or directory


In [21]:
ner_model = train_model(ner_config)

2020-07-29 14:41:13.76 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 101: [saving vocabulary to /root/.deeppavlov/models/ner_conll2003/word.dict]
2020-07-29 14:41:13.82 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 101: [saving vocabulary to /root/.deeppavlov/models/ner_conll2003/tag.dict]
2020-07-29 14:41:13.99 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 101: [saving vocabulary to /root/.deeppavlov/models/ner_conll2003/char.dict]
2020-07-29 14:41:13.101 INFO in 'deeppavlov.models.embedders.glove_embedder'['glove_embedder'] at line 52: [loading GloVe embeddings from `/root/.deeppavlov/downloads/embeddings/glove.6B.100d.txt`]



Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Use `tf.cast` instead.



2020-07-29 14:41:46.602 INFO in 'deeppavlov.core.layers.tf_layers'['tf_layers'] at line 760: 
Warning! tf.contrib.cudnn_rnn.CudnnCompatibleLSTMCell is used. It is okay for inference mode, but if you train your model with this cell it could NOT be used with tf.contrib.cudnn_rnn.CudnnLSTMCell later. 


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
seq_dim is deprecated, use seq_axis instead
Instructions for updating:
batch_dim is deprecated, use batch_axis instead


2020-07-29 14:41:46.685 INFO in 'deeppavlov.core.layers.tf_layers'['tf_layers'] at line 760: 
Warning! tf.contrib.cudnn_rnn.CudnnCompatibleLSTMCell is used. It is okay for inference mode, but if you train your model with this cell it could NOT be used with tf.contrib.cudnn_rnn.CudnnLSTMCell later. 
2020-07-29 14:41:48.786 DEBUG in 'deeppavlov.metrics.fmeasure'['fmeasure'] at line 397: processed 1424 tokens with 32 phrases; found: 971 phrases; correct: 0.

precision:  0.00%; recall:  0.00%; FB1:  0.00

	city: precision:  0.00%; recall:  0.00%; F1:  0.00 307

	cuisine: precision:  0.00%; recall:  0.00%; F1:  0.00 278

	date: precision:  0.00%; recall:  0.00%; F1:  0.00 25

	price_range: precision:  0.00%; recall:  0.00%; F1:  0.00 193

	restaurant_name: precision:  0.00%; recall:  0.00%; F1:  0.00 3

	time: precision:  0.00%; recall:  0.00%; F1:  0.00 165


2020-07-29 14:41:48.790 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 198: Initial best ner_f1 of 0


...

{"valid": {"eval_examples_count": 214, "metrics": {"ner_f1": 92.0635, "ner_token_f1": 92.9293}, "time_spent": "0:00:01"}}


2020-07-29 14:43:31.319 DEBUG in 'deeppavlov.metrics.fmeasure'['fmeasure'] at line 397: processed 1846 tokens with 50 phrases; found: 49 phrases; correct: 0.

precision:  83.67%; recall:  82.00%; FB1:  82.83

	city: precision:  91.67%; recall:  84.62%; F1:  88.00 12

	cuisine: precision:  92.86%; recall:  92.86%; F1:  92.86 14

	date: precision:  75.00%; recall:  50.00%; F1:  60.00 4

	price_range: precision:  100.00%; recall:  100.00%; F1:  100.00 3

	restaurant_name: precision:  0.00%; recall:  0.00%; F1:  0.00 0

	time: precision:  68.75%; recall:  84.62%; F1:  75.86 16


2020-07-29 14:43:31.562 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /root/.deeppavlov/models/ner_conll2003/word.dict]
2020-07-29 14:43:31.564 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /root/.deeppavlov/models/ner_conll2003/tag.dict]
2020-07-29 14:43:31.567 INFO in 'deeppavlov.core.data.simple_vocab'['simple

{"test": {"eval_examples_count": 267, "metrics": {"ner_f1": 82.8283, "ner_token_f1": 87.4372}, "time_spent": "0:00:01"}}


2020-07-29 14:44:04.630 INFO in 'deeppavlov.core.layers.tf_layers'['tf_layers'] at line 760: 
Warning! tf.contrib.cudnn_rnn.CudnnCompatibleLSTMCell is used. It is okay for inference mode, but if you train your model with this cell it could NOT be used with tf.contrib.cudnn_rnn.CudnnLSTMCell later. 
2020-07-29 14:44:04.709 INFO in 'deeppavlov.core.layers.tf_layers'['tf_layers'] at line 760: 
Warning! tf.contrib.cudnn_rnn.CudnnCompatibleLSTMCell is used. It is okay for inference mode, but if you train your model with this cell it could NOT be used with tf.contrib.cudnn_rnn.CudnnLSTMCell later. 
2020-07-29 14:44:06.139 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /root/.deeppavlov/models/ner_conll2003/model_no_pos]


INFO:tensorflow:Restoring parameters from /root/.deeppavlov/models/ner_conll2003/model_no_pos


Ner performs OK for everything except the restaurants' name identification.  
That's mostly because we avoid the usage of restaurants database here.

Let's check the NER performance running some slotfilling experiment

In [22]:
import json

json.dump(ner_config, open('ner_config.json', 'wt'))

In [23]:
import json
from deeppavlov import configs

from deeppavlov.core.common.file import read_json
from deeppavlov import build_model

slotfill_config = read_json(configs.ner.slotfill_dstc2)
slotfill_config['dataset_reader']['class_name'] = "md_yaml_dialogs_reader"
slotfill_config['metadata']['variables']['DATA_PATH'] = 'schema_resto_md_yaml'
slotfill_config['metadata']['variables']['SLOT_VALS_PATH'] = 'schema_resto_md_yaml/slotfill.json'
slotfill_config["chainer"]["pipe"][-1]["load_path"] = "schema_resto_md_yaml/slotfill.json"
slotfill_config["metadata"]["variables"]["NER_CONFIG_PATH"] = "ner_config.json"
slotfiller = build_model(slotfill_config, download=False)

2020-07-29 14:44:06.450 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /root/.deeppavlov/models/ner_conll2003/word.dict]
2020-07-29 14:44:06.453 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /root/.deeppavlov/models/ner_conll2003/tag.dict]
2020-07-29 14:44:06.454 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /root/.deeppavlov/models/ner_conll2003/char.dict]
2020-07-29 14:44:06.456 INFO in 'deeppavlov.models.embedders.glove_embedder'['glove_embedder'] at line 52: [loading GloVe embeddings from `/root/.deeppavlov/downloads/embeddings/glove.6B.100d.txt`]
2020-07-29 14:44:39.645 INFO in 'deeppavlov.core.layers.tf_layers'['tf_layers'] at line 760: 
Warning! tf.contrib.cudnn_rnn.CudnnCompatibleLSTMCell is used. It is okay for inference mode, but if you train your model with this cell it could NOT be used with tf.contrib.cudnn_rnn.CudnnLSTM

INFO:tensorflow:Restoring parameters from /root/.deeppavlov/models/ner_conll2003/model_no_pos


2020-07-29 14:44:41.274 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 45: Load path '/content/dp_big_demo_dir/schema_resto_md_yaml/slotfill.json' differs from save path '/root/.deeppavlov/models/slotfill_dstc2/model' in 'infer' mode for DstcSlotFillingNetwork.


In [24]:
slotfiller(["i'm looking for a thai food somewhere in SFO"])

[{'city': 'SFO', 'cuisine': 'Thai'}]

Seems OK. Let's save our slotfiller config and train and evaluate the restaurants bot, finally.

In [25]:
import json

json.dump(slotfill_config, open('slotfill_config.json', 'wt'))

In [26]:
from deeppavlov import configs
from deeppavlov.core.common.file import read_json


gobot_config = read_json(configs.go_bot.gobot_md_yaml_minimal)
gobot_config['chainer']['pipe'][-1]['slot_filler'] = {"config_path": "slotfill_config.json"}

gobot_config['metadata']['variables']['DATA_PATH'] = 'schema_resto_md_yaml'
gobot_config['metadata']['variables']['MODEL_PATH'] = '.'

In [27]:
!cp schema_resto_md_yaml/stories.md schema_resto_md_yaml/stories-trn.md
!cp schema_resto_md_yaml/stories.md schema_resto_md_yaml/stories-tst.md
!cp schema_resto_md_yaml/stories.md schema_resto_md_yaml/stories-val.md

In [28]:
!echo "{}" > schema_resto_md_yaml/dstc2-actions2slots.json  # a hack that will be redundant in future versions. keep an empty json file in the data folder

In [29]:
from deeppavlov import train_model

train_model(gobot_config, download=False);

2020-07-29 14:44:46.811 INFO in 'deeppavlov.dataset_readers.md_yaml_dialogs_reader'['md_yaml_dialogs_reader'] at line 123: INSIDE MLU_MD_DialogsDatasetReader._read_domain_knowledge(): reading domain knowledge from /content/dp_big_demo_dir/schema_resto_md_yaml/domain.yml
2020-07-29 14:44:46.978 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from /tmp/tmpegk9s_lo]
2020-07-29 14:44:47.80 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from /tmp/tmpp4sn4fbf]
2020-07-29 14:44:47.187 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from /tmp/tmphy0gjblt]
2020-07-29 14:44:47.206 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 101: [saving vocabulary to /content/dp_big_demo_dir/word.dict]
2020-07-29 14:44:47.209 INFO in 'deeppavlov.dataset_readers.md_yaml_dialogs_reader'['md_yaml_dialogs_reader'] at line 123: INSIDE MLU_MD_DialogsDa

INFO:tensorflow:Restoring parameters from /root/.deeppavlov/models/ner_conll2003/model_no_pos


2020-07-29 14:45:22.892 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 45: Load path '/content/dp_big_demo_dir/schema_resto_md_yaml/slotfill.json' differs from save path '/root/.deeppavlov/models/slotfill_dstc2/model' in 'infer' mode for DstcSlotFillingNetwork.
2020-07-29 14:45:22.894 INFO in 'deeppavlov.models.embedders.glove_embedder'['glove_embedder'] at line 52: [loading GloVe embeddings from `/root/.deeppavlov/downloads/embeddings/glove.6B.100d.txt`]
2020-07-29 14:45:56.593 INFO in 'deeppavlov.models.go_bot.policy.policy_network'['policy_network'] at line 89: INSIDE PolicyNetwork init(). Initializing PolicyNetwork from scratch.
2020-07-29 14:45:59.725 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 198: Initial best per_item_action_accuracy of 0.0024


{"valid": {"eval_examples_count": 85, "metrics": {"per_item_action_accuracy": 0.0024}, "time_spent": "0:00:04", "epochs_done": 0, "batches_seen": 0, "train_examples_seen": 0, "impatience": 0, "patience_limit": 10}}
{"train": {"eval_examples_count": 85, "metrics": {"per_item_action_accuracy": 0.2799}, "time_spent": "0:00:09", "epochs_done": 0, "batches_seen": 15, "train_examples_seen": 60, "learning_rate": 0.003, "momentum": 0.95, "loss": 2.740983168284098}}


2020-07-29 14:46:07.693 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 206: Improved best per_item_action_accuracy of 0.2799
2020-07-29 14:46:07.697 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 208: Saving model
2020-07-29 14:46:07.698 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 75: [saving model to /content/dp_big_demo_dir/model/policy]


...

{"test": {"eval_examples_count": 85, "metrics": {"per_item_action_accuracy": 0.7333}, "time_spent": "0:00:03"}}


2020-07-29 14:52:55.417 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from /tmp/tmpb30b43mx]
2020-07-29 14:52:55.521 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from /tmp/tmpe1awpj25]
2020-07-29 14:52:55.624 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from /tmp/tmpyajukubt]
2020-07-29 14:52:55.810 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /root/.deeppavlov/models/ner_conll2003/word.dict]
2020-07-29 14:52:55.812 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /root/.deeppavlov/models/ner_conll2003/tag.dict]
2020-07-29 14:52:55.815 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /root/.deeppavlov/models/ner_conll2003/char.dict]
2020-07-29 14:52:55.818 INFO in 'deeppavlov.models.embed

INFO:tensorflow:Restoring parameters from /root/.deeppavlov/models/ner_conll2003/model_no_pos


2020-07-29 14:53:30.573 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 45: Load path '/content/dp_big_demo_dir/schema_resto_md_yaml/slotfill.json' differs from save path '/root/.deeppavlov/models/slotfill_dstc2/model' in 'infer' mode for DstcSlotFillingNetwork.
2020-07-29 14:53:30.575 INFO in 'deeppavlov.models.embedders.glove_embedder'['glove_embedder'] at line 52: [loading GloVe embeddings from `/root/.deeppavlov/downloads/embeddings/glove.6B.100d.txt`]
2020-07-29 14:54:04.568 INFO in 'deeppavlov.models.go_bot.policy.policy_network'['policy_network'] at line 86: INSIDE PolicyNetwork init(). Initializing PolicyNetwork from checkpoint.
2020-07-29 14:54:04.573 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /content/dp_big_demo_dir/model/policy]


INFO:tensorflow:Restoring parameters from /content/dp_big_demo_dir/model/policy


In [30]:
bot = build_model(gobot_config)

2020-07-29 14:54:04.719 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /content/dp_big_demo_dir/word.dict]
2020-07-29 14:54:04.722 INFO in 'deeppavlov.dataset_readers.md_yaml_dialogs_reader'['md_yaml_dialogs_reader'] at line 123: INSIDE MLU_MD_DialogsDatasetReader._read_domain_knowledge(): reading domain knowledge from /content/dp_big_demo_dir/schema_resto_md_yaml/domain.yml
2020-07-29 14:54:04.892 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from /tmp/tmpzt7zsk9v]
2020-07-29 14:54:04.996 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from /tmp/tmpk7sfljqx]
2020-07-29 14:54:05.108 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from /tmp/tmp33edcyxb]
2020-07-29 14:54:05.297 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /root/.deeppavlov/mo

INFO:tensorflow:Restoring parameters from /root/.deeppavlov/models/ner_conll2003/model_no_pos


2020-07-29 14:54:39.528 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 45: Load path '/content/dp_big_demo_dir/schema_resto_md_yaml/slotfill.json' differs from save path '/root/.deeppavlov/models/slotfill_dstc2/model' in 'infer' mode for DstcSlotFillingNetwork.
2020-07-29 14:54:39.530 INFO in 'deeppavlov.models.embedders.glove_embedder'['glove_embedder'] at line 52: [loading GloVe embeddings from `/root/.deeppavlov/downloads/embeddings/glove.6B.100d.txt`]
2020-07-29 14:55:13.261 INFO in 'deeppavlov.models.go_bot.policy.policy_network'['policy_network'] at line 86: INSIDE PolicyNetwork init(). Initializing PolicyNetwork from checkpoint.
2020-07-29 14:55:13.266 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /content/dp_big_demo_dir/model/policy]


INFO:tensorflow:Restoring parameters from /content/dp_big_demo_dir/model/policy


In [37]:
bot.reset()

bot(["Hey!"])[0][0].actions_tuple

('utter_hi',)

Ok the bot performs OK on greeting

In [38]:
bot(["I'd like to find a restaurant for this evening"])[0][0].actions_tuple

('utter_REQUEST_Cuisine',)

The bot asks for an inevitably necessary information.

In [39]:
bot(["Somewhere in Oakland, sushi and for two people please"])[0][0].actions_tuple

('utter_OFFER_RestaurantName', 'OFFER_City')

And so on.

In [40]:
bot(["Cool! That's what I was looking for, thanks!"])[0][0].actions_tuple

('utter_REQUEST_Time',)

Let's say goodbye to our bot

In [35]:
bot(["Bye bot"])[0][0].actions_tuple

('utter_REQUEST_Time',)

After the goodbye bot does perform uninterpretable. That's what most of recurrent system do after the end of the sequence cause no one reaches that poing. 

In [36]:
bot(["Bye bot"])[0][0].actions_tuple

('start',)

## Final words

More of in-depth examples of goal-oriented bots and their features are coming with future releases, stay tuned!